In [17]:
import numpy as np
import pandas as pd
df = pd.read_csv('Final.csv')
df.head()

,Normalized_Price,Rating,city,Kilometers,Owner,Fuel_Type,Transmission,Car_Age,Is_Insurance_Valid,Price_per_Kilometer,Insurance_and_Age
0,0.016173,4.0,3686,0.028231,1,3,1,0.250,0,5.012308,0
1,0.030921,4.2,3686,0.032027,3,3,1,0.175,0,8.446903,0
2,0.032698,4.3,3686,0.018818,3,3,1,0.150,0,15.202471,0
3,0.022024,4.4,3686,0.022475,1,3,1,0.175,0,8.573555,0
4,0.027838,4.4,3686,0.009912,3,3,1,0.075,0,24.570927,0


In [18]:
df.columns

Index(['Normalized_Price', 'Rating', 'city', 'Kilometers', 'Owner',
       'Fuel_Type', 'Transmission', 'Car_Age', 'Is_Insurance_Valid',
       'Price_per_Kilometer', 'Insurance_and_Age'],
      dtype='object')

In [19]:
X = df[['Rating', 'city', 'Kilometers', 'Owner', 'Fuel_Type',
    'Transmission', 'Car_Age', 'Is_Insurance_Valid', 'Price_per_Kilometer',
    'Insurance_and_Age']] 
y = df['Normalized_Price'] # Dependent Variable

Best Random State

In [20]:
Train = []
CV = []

for i in range (0,100):
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state = i)
    
    from sklearn.linear_model import LinearRegression
    model = LinearRegression()
    model.fit(X_train,y_train)
    
    ypred_train = model.predict(X_train)
    
    from sklearn.metrics import r2_score
    Train.append(r2_score(y_train,ypred_train))
    
    from sklearn.model_selection import cross_val_score
    CV.append(cross_val_score(model,X_train,y_train,cv=5).mean())

em = pd.DataFrame({'Train':Train,'CV':CV})
gm = em[(abs(em['Train']-em['CV'])<=0.05)]
rs = gm [gm['CV']==gm['CV'].max()].index.tolist()[0]
print('Best Random State Number :',rs)

Best Random State Number : 73


# **Train Test split**

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state = 73)


# **MODELLING**

# Multiple Linear Regression

In [22]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train,y_train)


LinearRegression()

In [23]:
# Predict on Train Data
ypred_train = model.predict(X_train)

# Evaluate the model on the train data
from sklearn.metrics import r2_score
print('Train R2 :',r2_score(y_train,ypred_train))

# Cross Validation Score
from sklearn.model_selection import cross_val_score
print ('Cross Validation Score :',cross_val_score(model,X_train,y_train,cv=10).mean())

Train R2 : 0.40302782469474563
Cross Validation Score : 0.4033516726994453


In [24]:
# Evaluate the model on test data
ypred_test =model.predict(X_test)
from sklearn.metrics import r2_score
print('Test R2 :',r2_score(y_test,ypred_test))

Test R2 : 0.3519228731306543


# Polynomial Regression (Non-Linear Regression)

In [25]:
from sklearn.preprocessing import PolynomialFeatures
polynomial_converter = PolynomialFeatures(degree = 3)
X_train_poly=pd.DataFrame(polynomial_converter.fit_transform(X_train))


#modelling on train data 
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train_poly,y_train)


#prediction on train data 
ypred_train = model.predict(X_train_poly)

# Evaluation on train data
from sklearn.metrics import mean_squared_error,r2_score
print("train RMSE:",np.sqrt(mean_squared_error(y_train,ypred_train)))
print("Train R2 :",r2_score(y_train,ypred_train))


#cross validation on train data
print("cross valiodation score:",cross_val_score(model,X_train_poly,y_train,cv=10).mean())

train RMSE: 0.03383893254281124
Train R2 : 0.3203243039339114
cross valiodation score: -2802174.0223018536


In [26]:
#data preprocessing on test data 
X_test_poly = pd.DataFrame(polynomial_converter.transform(X_test))

#prediction on test data 
ypred_test = model.predict(X_test_poly)

#evaluation on test data 
print("test RMSE:",np.sqrt(mean_squared_error(y_test,ypred_test)))
print("Test R2 :",r2_score(y_test,ypred_test))

test RMSE: 0.3203562839582437
Test R2 : -49.76401266472322


# Ridge Regression Model

In [27]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
estimator = Ridge()
# Parameters & Values
param_grid = {'alpha' :list(range(1,100))}
# Identifying the best hyperparameter the the data
model_hp = GridSearchCV(estimator, param_grid, cv=10,scoring='r2')

model_hp.fit(X_train,y_train)
model_hp.best_params_

{'alpha': 1}

**Modelling Ridge Regression using best Hyperparameter**

In [28]:
# Modelling 
ridge_best = Ridge(alpha=9)
ridge_best.fit(X_train,y_train)

print("Intercept :",ridge_best.intercept_)
print("Coefficients :",ridge_best.coef_)

Intercept : 0.06105495595704105
Coefficients : [ 1.06469002e-02  2.03356988e-06 -6.26140620e-02 -1.11000142e-03
 -6.62601971e-03 -3.58316902e-02 -9.07509576e-02  2.09792046e-02
  3.59041087e-08 -1.73095610e-03]


Evalution

In [29]:
ypred_train = ridge_best.predict(X_train)
print("Train R2 :",r2_score(y_train,ypred_train))
print("cross valiodation score:",cross_val_score(ridge_best,X_train,y_train,cv=5).mean())

Train R2 : 0.4027582483320562
cross valiodation score: 0.4027707524452545


In [30]:
ypred_test =ridge_best.predict(X_test)
from sklearn.metrics import r2_score
print('Test R2 :',r2_score(y_test,ypred_test))

Test R2 : 0.35206735868116334


#  Lasso Regression

In [31]:
from sklearn.model_selection import GridSearchCV

# MODEL
from sklearn.linear_model import Lasso
estimator = Lasso()

param_grid = {'alpha' :list(range(1,100))}
# Identifying the best hyperparameter the the data
model_hp = GridSearchCV(estimator, param_grid, cv=10,scoring='r2')

model_hp.fit(X_train,y_train)
model_hp.best_params_

{'alpha': 1}

In [32]:
# Modelling 
from sklearn.linear_model import Lasso
lasso_best = Lasso(alpha=0.01)
lasso_best.fit (X_train, y_train)

# Predicting
ypred_train = lasso_best.predict(X_train)

from sklearn.metrics import r2_score
print("Train R2 :",r2_score(y_train,ypred_train))

from sklearn.model_selection import cross_val_score
print ("CV Score :",cross_val_score(lasso_best,X_train,y_train,cv=5).mean())

# prediction & Evaluation on test data 

ypred_test = lasso_best.predict (X_test)
print("Test R2 :",r2_score(y_test,ypred_test))

Train R2 : 0.09552744902684052
CV Score : 0.0952053821893494
Test R2 : 0.08334707018993914


# Random Forest

In [33]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score

# Initialize the RandomForestRegressor model
rf_model = RandomForestRegressor()

# Define the hyperparameter grid for Random Forest
rf_param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
}

# Perform GridSearchCV for Random Forest
rf_grid_search = GridSearchCV(estimator=rf_model, param_grid=rf_param_grid, scoring='neg_mean_squared_error', cv=3)
rf_grid_search.fit(X_train, y_train)

# Best parameters and score for Random Forest
print(f"Random Forest Best Params: {rf_grid_search.best_params_}")
print(f"Random Forest Best Score: {rf_grid_search.best_score_}")

# Evaluate on test set
rf_best_model = rf_grid_search.best_estimator_
rf_y_pred = rf_best_model.predict(X_test)
rf_r2 = r2_score(y_test, rf_y_pred)
print(f"Random Forest R² Score: {rf_r2}")

Random Forest Best Params: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 50}
Random Forest Best Score: -4.911030906111043e-05
Random Forest R² Score: 0.9361419196091119


# Gradient Boosting 

In [37]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
# Initialize and train the model
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=45)
gb_model.fit(X_train, y_train)

# Predict car prices
y_pred = gb_model.predict(X_test)
# Evaluation
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R2 Score: {r2}")

Mean Absolute Error (MAE): 0.0039610946668302595
Mean Squared Error (MSE): 0.0001309197794391775
R2 Score: 0.9352418081438457


# SVR

In [38]:
from sklearn.svm import SVR

# Initialize and train the model
svr_model = SVR(kernel='rbf')
svr_model.fit(X_train, y_train)

# Predict car prices
y_pred = svr_model.predict(X_test)
# Evaluation
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R2 Score: {r2}")

Mean Absolute Error (MAE): 0.06747039379728617
Mean Squared Error (MSE): 0.005523960684741762
R2 Score: -1.7323732698048881


# XG Boost

In [39]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score

# Initialize the XGBoost Regressor
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)

# Define the hyperparameter grid for XGBoost
xgb_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

# Perform GridSearchCV for XGBoost
xgb_grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=xgb_param_grid,
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=1
)
xgb_grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 108 candidates, totalling 324 fits


GridSearchCV(cv=3,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=42, ...),
             param_grid={'colsample_bytree': [0.8, 1.0],
                         'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [100, 200, 300],
                         'subsample': [0.8, 1.0]},
             scoring='neg_mean_squared_error', verbose=1)

In [50]:
# Best parameters and score for XGBoost
print(f"XGBoost Best Params: {xgb_grid_search.best_params_}")
print(f"XGBoost Best Score: {xgb_grid_search.best_score_}")

# Evaluate on the train set
xgb_best_model = xgb_grid_search.best_estimator_
xgb_y_pred = xgb_best_model.predict(X_train)
xgb_r2 = r2_score(y_train, xgb_y_pred)
print(f"XGBoost train R² Score: {xgb_r2}")

# Evaluate on the test set
xgb_best_model = xgb_grid_search.best_estimator_
xgb_y_pred = xgb_best_model.predict(X_test)
xgb_r2 = r2_score(y_test, xgb_y_pred)
print(f"XGBoost test R² Score: {xgb_r2}")

XGBoost Best Params: {'colsample_bytree': 1.0, 'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 300, 'subsample': 1.0}
XGBoost Best Score: -3.3361357542668325e-05
XGBoost train R² Score: 0.9907706812323388
XGBoost Cross-Validation Scores:  [-4.94601881e-05 -5.17176823e-05 -2.97449331e-05 -4.31793357e-05
 -6.21490822e-05]
XGBoost test R² Score: 0.9650990707853064


# KNN

In [42]:
from sklearn.neighbors import KNeighborsRegressor

# Initialize and train the model
knn_model = KNeighborsRegressor(n_neighbors=5)
knn_model.fit(X_train, y_train)

# Predict car prices
y_pred = knn_model.predict(X_test)
# Evaluation
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R2 Score: {r2}")


Mean Absolute Error (MAE): 0.01478189517178911
Mean Squared Error (MSE): 0.0011813393439786038
R2 Score: 0.4156620167533148


# Decision Tree 

In [43]:
from sklearn.tree import DecisionTreeRegressor

# Initialize and train the model
dt_model = DecisionTreeRegressor(random_state=42)
dt_model.fit(X_train, y_train)

# Predict car prices
y_pred = dt_model.predict(X_test)
# Evaluation
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R2 Score: {r2}")

Mean Absolute Error (MAE): 0.0018387001755804332
Mean Squared Error (MSE): 0.0001425916042307473
R2 Score: 0.9294684538623024


# ANN

In [44]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam


In [45]:
# Initialize the ANN
ann_model = Sequential()

# Input layer
ann_model.add(Dense(units=64, activation='relu', input_dim=X_train.shape[1]))

# Hidden layers
ann_model.add(Dense(units=64, activation='relu'))
ann_model.add(Dense(units=32, activation='relu'))

# Output layer (single unit for regression)
ann_model.add(Dense(units=1))

# Compile the model
ann_model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')


c:\Users\darsh\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [46]:
# Train the model
history = ann_model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)


Epoch 1/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 2s 890us/step - loss: 593814.9375 - val_loss: 3480.0012
Epoch 2/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 0s 637us/step - loss: 16526.4727 - val_loss: 238.5547
Epoch 3/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 0s 634us/step - loss: 372.1546 - val_loss: 3544.4419
Epoch 4/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 0s 702us/step - loss: 74033.9453 - val_loss: 50773.5352
Epoch 5/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 1s 841us/step - loss: 168428.7031 - val_loss: 518.2092
Epoch 6/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 1s 758us/step - loss: 30566.5977 - val_loss: 399.6135
Epoch 7/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 0s 682us/step - loss: 201.6756 - val_loss: 421280.2500
Epoch 8/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 0s 631us/step - loss: 135588.8906 - val_loss: 3.8101
Epoch 9/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 0s 634us/step - loss: 1420.4930 - val_loss: 9.9990
Epoch 10/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 0s 667us/step - loss: 138.9242 - val_loss: 1.1348
Epoch 11/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 0s 634us/s

In [47]:
# Evaluate the model on test data
test_loss = ann_model.evaluate(X_test, y_test)
print(f"Test Loss (MSE): {test_loss}")

# Make predictions on the test set
y_pred = ann_model.predict(X_test)

# Example: Compare actual vs predicted prices
comparison = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred.flatten()})
print(comparison.head())


201/201 ━━━━━━━━━━━━━━━━━━━━ 0s 498us/step - loss: 0.0023
Test Loss (MSE): 0.0018190228147432208
201/201 ━━━━━━━━━━━━━━━━━━━━ 0s 477us/step
         Actual  Predicted
4310   0.046289   0.047801
14577  0.148125   0.036815
11086  0.021293   0.040340
2502   0.064805   0.046977
16287  0.014350   0.038460


In [48]:
# Predict the car prices on the test set
y_pred = ann_model.predict(X_test)

# Calculate the R² score
r2 = r2_score(y_test, y_pred)

print(f"R² Score: {r2}")

201/201 ━━━━━━━━━━━━━━━━━━━━ 0s 603us/step
R² Score: 0.10023790381589603


# **Save the Best Model**

In [51]:
import joblib

joblib.dump(xgb_best_model, 'xgboost_model.pkl')


['xgboost_model.pkl']